## 3. Semantic Search - Vectorize search (FAQs)

Semantic Search - ingest/search

In [1]:
from dotenv import load_dotenv
load_dotenv("../.env")

True

In [2]:
from azure.core.credentials import AzureKeyCredential  
from azure.search.documents import SearchClient  
from azure.search.documents.indexes import SearchIndexClient  
from azure.search.documents.indexes.models import (  
    SearchIndex,  
    SearchField,  
    SearchFieldDataType,  
    SimpleField,  
    SearchableField,  
    SearchIndex,  
    SemanticConfiguration,  
    SearchField,  
    VectorSearch,
    SemanticSearch,
    SemanticPrioritizedFields,
    SemanticField,
    HnswAlgorithmConfiguration,
    HnswParameters,
    VectorSearchAlgorithmMetric,
    VectorSearchProfile,
    AzureOpenAIVectorizer,
    AzureOpenAIVectorizerParameters
)

In [3]:
import os
service_endpoint = os.getenv("AZSCH_ENDPOINT")  
credential = AzureKeyCredential(os.environ["AZSCH_KEY"])

print(service_endpoint)

api_key = os.getenv("AZURE_OPENAI_KEY")
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")

https://iksch.search.windows.net


In [4]:
# Create a search index
def create_search_index(index_name, model_name="text-embedding-3-large"):
    index_client = SearchIndexClient(
        endpoint=service_endpoint, credential=credential)
    fields = [
        SimpleField(name="id", type=SearchFieldDataType.String, key=True),
        SearchableField(name="title", type=SearchFieldDataType.String,
                        searchable=True, retrievable=True,
                        analyzer_name="ko.microsoft"),
        SearchableField(name="chunk", type=SearchFieldDataType.String,
                        searchable=True, retrievable=True,
                        analyzer_name="ko.microsoft"),
        SearchableField(name="category", type=SearchFieldDataType.String,
                        searchable=False, retrievable=True,
                        facetable=True, filterable=True),
        SearchableField(name="type", type=SearchFieldDataType.String,
                        searchable=False, retrievable=True,
                        facetable=True, filterable=True),
        SimpleField(name="parent_id", type=SearchFieldDataType.String),
        SimpleField(name="chunk_id", type=SearchFieldDataType.String),
        SearchField(name="vector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
                    searchable=True, vector_search_dimensions=3072,
                    vector_search_profile_name="myHnswProfile")
    ]

    vector_search = VectorSearch(  
        algorithms=[  
            HnswAlgorithmConfiguration(  
                name="myHnsw",  
                parameters=HnswParameters(  
                    m=4,  
                    ef_construction=400,  
                    ef_search=500,  
                    metric=VectorSearchAlgorithmMetric.COSINE,  
                ),  
            )
        ],  
        profiles=[  
            VectorSearchProfile(  
                name="myHnswProfile",  
                algorithm_configuration_name="myHnsw",
                vectorizer_name="vectorizer"
            )
        ],
        vectorizers=[
            AzureOpenAIVectorizer(
                vectorizer_name="vectorizer",
                kind="azureOpenAI",
                parameters = AzureOpenAIVectorizerParameters(
                    resource_url=azure_endpoint,
                    api_key=api_key,
                    deployment_name=model_name,
                    model_name=model_name
                ),
            )
        ]
    )  

    semantic_config = SemanticConfiguration(  
        name="semantic-config",  
        prioritized_fields=SemanticPrioritizedFields(  
            title_field=SemanticField(field_name="title"),
            content_fields=[SemanticField(field_name="chunk")]  
        )
    )

    # Create the semantic search with the configuration  
    semantic_search = SemanticSearch(configurations=[semantic_config]) 

    # Create the search index
    index = SearchIndex(name=index_name, fields=fields,
                        vector_search=vector_search, semantic_search=semantic_search)
    result = index_client.create_or_update_index(index)
    print(f' {result.name} created')

In [5]:
index_name = "ncfaqs-kor-index"
create_search_index(index_name, 'text-embedding-3-large')

 ncfaqs-kor-index created


In [6]:
search_client = SearchClient(endpoint=service_endpoint, index_name=index_name, credential=credential)

In [7]:
import pandas as pd
from tqdm import tqdm

In [8]:
df = pd.read_pickle('./sample_faqs_chunked.pkl')
df.head()

,title,chunk,category,parent_id,chunk_id,embeddings
0,계정 인증을 해제하고 싶어요.,계정 인증 해제는 아래의 방법을 참고해 주시길 바랍니다.\n■ 계정 인증 해제 방법...,계정,469829,0,"[-0.009334231726825237, -0.07371649891138077, ..."
1,밤까마귀 장비를 강화했는데 이벤트 및 업적 카운트에 반영되지 않아요.,"강화를 시도해도 파괴되지 않는 '보호의 밤까마귀 강화 주문서'로 강화했을 경우, 이...",게임,412428,0,"[-0.0031998935155570507, -0.034888189285993576..."
2,"웹 상점 - 결제 취소는 되었지만, 결제 금액이 환불되지 않았어요.",결제 수단인 카드사 혹은 결제사의 정책에 따라 최종 결제 환불까지 영업일 기준 1~...,결제(PC),398796,0,"[-0.032882701605558395, -0.028877221047878265,..."
3,웹 상점 - 결제 수단 별 한도가 궁금해요.,결제 수단 별 한도는 각 카드사 및 결제사의 정책에 따라 변동될 수 있습니다.\n▶...,결제(PC),398795,0,"[-0.03583231195807457, -0.005664754193276167, ..."
4,웹 상점 - 상품을 구매했지만 보관함에 들어오지 않았어요.,"웹 상점에서 상품을 구매하면, 공식 사이트 내 대표 캐릭터로 설정된 캐릭터에게 지급...",결제(PC),398794,0,"[-0.030095618218183517, -0.008123816922307014,..."


In [9]:
count = 0
batch_size = 20
for i in tqdm(range(0, len(df), batch_size)):
    # set end position of batch
    i_end = min(i+batch_size, len(df))
    
    documents = df[i:i_end].apply(
        lambda row: {'id': str(row.name), 
                     'title': row['title'], 
                     'chunk': row['chunk'],
                     'category': row['category'],
                     'type': 'faq',
                     #'vector': generate_embeddings(row['chunk'])
                     'parent_id': str(row['parent_id']),
                     'chunk_id': str(row['chunk_id']),
                     'vector': row['embeddings']
                    }, axis=1).to_list()
    
    result = search_client.upload_documents(documents)  

100%|██████████| 20/20 [00:14<00:00,  1.34it/s]


### Vector Search

In [10]:
from azure.search.documents.models import (
    VectorizableTextQuery,
    VectorQuery,
    VectorizedQuery,
    QueryType,
    QueryCaptionType,
    QueryAnswerType)

In [11]:
def azsch_vector_query(query):
    #vector_query = VectorizedQuery(vector=generate_embeddings(query), k_nearest_neighbors=3, fields="vector")
    vector_query = VectorizableTextQuery(text=query, k_nearest_neighbors=50, fields="vector", exhaustive=True)

    results = search_client.search(  
        search_text=None,  
        vector_queries=[vector_query],
        select=["title", "chunk", "category", "type", "parent_id"],
        query_language="ko-kr",
        top=10 # for limiting text search
    ) 

    for i, result in enumerate(results, 1): 
        print(f"{i}: {result['@search.score']:.10f}: {result['parent_id']}, {result['title']}, {result['category']}")  

In [12]:
azsch_vector_query('해외에서도 사용한가요?')

1: 0.6034120300: 8, 나이트크로우를 해외에서 플레이할 수 있나요?, 설치/실행(MO)
2: 0.5733986000: 93, 구글 연동 계정 로그인 시, "Google에서 이 계정이 본인 계정이라는 것을 확인하지 못했습니다."라는 메시지와 함께 로그인이 불가합니다., 계정
3: 0.5728779000: 20, 상품 결제 중 오류가 발생했어요., 결제(MO)
4: 0.5709528300: 117, 이용 중인 계정이 환불 제재 되었어요., 계정
5: 0.5664265000: 19, 같은 상품인데 스토어별 구매 가격이 다릅니다., 결제(MO)
6: 0.5629176500: 25, PC에서 문의/신고 글 등록이 되지 않아요., 건의/신고
7: 0.5620746600: 398794, 웹 상점 - 상품을 구매했지만 보관함에 들어오지 않았어요., 결제(PC)
8: 0.5613286000: 63, 결제 내역(영수증)은 어디서 확인하나요?, 결제(MO)
9: 0.5591763000: 229992, 상품 구매에 실패하였습니다.(P:-477) 에러가 발생해요., 결제(PC)
10: 0.5584575000: 116, 기기 변경 또는 재설치한 후에도 이용하던 게임 정보를 불러올 수 있나요?, 계정


### Hybrid Search

In [13]:
def azsch_hybrid_query(query):
    #vector_query = VectorizedQuery(vector=generate_embeddings(query), k_nearest_neighbors=3, fields="vector")
    vector_query = VectorizableTextQuery(text=query, k_nearest_neighbors=50, fields="vector", exhaustive=True)

    results = search_client.search(  
        search_text=query,  
        vector_queries=[vector_query],
        select=["title", "chunk", "category", "type", "parent_id"],
        query_language="ko-kr",
        top=10 # for limiting text search
    ) 

    for i, result in enumerate(results, 1): 
        print(f"{i}: {result['@search.score']:.10f}: {result['parent_id']}, {result['title']}, {result['category']}")  

In [14]:
azsch_hybrid_query('해외에서도 사용한가요?')

1: 0.0333333351: 8, 나이트크로우를 해외에서 플레이할 수 있나요?, 설치/실행(MO)
2: 0.0325224735: 20, 상품 결제 중 오류가 발생했어요., 결제(MO)
3: 0.0278097428: 60, 상품을 환불하고 싶어요., 결제(MO)
4: 0.0275297612: 122, 단말기 설정을 무음으로 했는데도 게임 소리가 들려요., 시스템(MO)
5: 0.0274436101: 398796, 웹 상점 - 결제 취소는 되었지만, 결제 금액이 환불되지 않았어요., 결제(PC)
6: 0.0270319637: 74, 아이템 버리기는 어떤 기능인가요?, 게임
7: 0.0270319637: 14, 네트워크 오류로 이용 중 접속이 끊기는 현상이 발생합니다., 시스템(MO)
8: 0.0268301349: 63, 결제 내역(영수증)은 어디서 확인하나요?, 결제(MO)
9: 0.0267857146: 9, 단말기 OS 정보는 어떻게 확인할 수 있나요?, 설치/실행(MO)
10: 0.0266205706: 38231, 밤까마귀 장비 세공 확률이 궁금해요., 게임


### Semantic rerank

combination of `title(description)` text + `meaning` text + `meaning` vector search result

In [15]:
def azsch_rerank_query(query):
    #vector_query = VectorizedQuery(vector=generate_embeddings(query), k_nearest_neighbors=3, fields="vector")
    vector_query = VectorizableTextQuery(text=query, k_nearest_neighbors=50, fields="vector", exhaustive=True)

    results = search_client.search(  
        search_text=query,  
        vector_queries=[vector_query],
        select=["title", "chunk", "category", "type", "parent_id"],
        query_type=QueryType.SEMANTIC,
        semantic_configuration_name='semantic-config',
        query_caption=QueryCaptionType.EXTRACTIVE,
        #query_answer=QueryAnswerType.EXTRACTIVE,
        query_language="ko-kr",
        top=10 # for limiting text search
    ) 

    for result in results:  
        if result["@search.captions"]:
            caption = result["@search.captions"][0]
            print(f"{result['@search.reranker_score']:.5f}/{result['@search.score']:.5f}: {result['parent_id']}, {result['title']}")
            print(f"\t{caption.highlights}")  
        else:
            print(f"{result['@search.reranker_score']:.5f}/{result['@search.score']:.5f}: {result['parent_id']}, {result['title']}")  
 

In [16]:
azsch_rerank_query('해외에서도 사용한가요?')

2.49927/0.03333: 8, 나이트크로우를 해외에서 플레이할 수 있나요?
	<em>현재 나이트크로우는 국내 서비스만을 지원하고 있습니다.</em> 그러므로<em> 해외에서 다운로드 및 플레이는 어려울 수 있는 점 양해 부탁드립니다.</em>
2.48793/0.03252: 20, 상품 결제 중 오류가 발생했어요.
	결제 오류가 발생하는 대표적인 사례는 다음과 같습니다. 1. 연결된 네트워크/통신 환경이 불안정한 경우- 통신 환경이 원활한 곳에서 게임을 이용해주세요. ※ 다수의 인원이 모여있는 환경에서 Wi-fi를 이용하신다면 LTE 등의 데이터 환경으로 전환하여 진행해주시기 바랍니다. 2. 결제 수단으로 연결한 카드 정보가 비정상적으로 인식되는 경우- 등록된 카드가 아닌 다른 카드로의 등록 및 사용해주세요. 3. <em>해외 결제 신용카드가 아닌 경우- 안정적인 결제가 가능하도록 해외 결제가 가능한 신용카드로 등록 및 사용해주세요.</em> 4. 이용 한도가 초과되었을 경우- 카드 이용한도 증액을 요청하시거나 통신사의 결제 내용을 확인해주세요..
1.88515/0.01149: 140, 나이트크로우에 접속할 수 없어요.
	나이트크로<em>우는 온라인 환경에서 이용할 수 있는 </em>멀티<em> 플랫폼 게임이므</em>로,네트워크 환경이 좋지 않을 경우 게임 이용이 불가하거나 끊기는 현상이 발생할 수 있습니다. 쾌적한 게임 이용을 위해 통신 상태가 원활한 곳에서 플레이 해주시기 바랍니다.
1.75932/0.01538: 25, PC에서 문의/신고 글 등록이 되지 않아요.
	PC 환경에서<em> 인터넷 익스플로러(Internet Explorer) 브라우저를</em><em> 통하여 문의를 남겨주실 경우 정상적인 이용이 불가할 수 있습니다.</em> 이 경우<em> 구글 크롬(Chorme) 브라우저를 통하여 고객센터 문의 접수를 권장드립니다.</em>
1.64447/0.01613: 18, 물약을 구매하였는데 사용할 수 없어요.
	나이트크로우의<em> 체력 회복 물

In [17]:
azsch_rerank_query('해외에서도 접속 가능한가요?')

2.47357/0.03333: 8, 나이트크로우를 해외에서 플레이할 수 있나요?
	<em>현재 나이트크로우는 국내 서비스만을 지원하고 있습니다.</em> 그러므로<em> 해외에서 다운로드 및 플레이는 어려울 수 있는 점 양해 부탁드립니다.</em>
2.20843/0.03102: 20, 상품 결제 중 오류가 발생했어요.
	결제 오류가 발생하는 대표적인 사례는 다음과 같습니다. 1. 연결된 네트워크/통신 환경이 불안정한 경우- 통신 환경이 원활한 곳에서 게임을 이용해주세요. ※ 다수의 인원이 모여있는 환경에서 Wi-fi를 이용하신다면 LTE 등의 데이터 환경으로 전환하여 진행해주시기 바랍니다. 2. 결제 수단으로 연결한 카드 정보가 비정상적으로 인식되는 경우- 등록된 카드가 아닌 다른 카드로의 등록 및 사용해주세요. 3. <em>해외 결제 신용카드가 아닌 경우- 안정적인 결제가 가능하도록 해외 결제가 가능한 신용카드로 등록 및 사용해주세요.</em> 4. 이용 한도가 초과되었을 경우- 카드 이용한도 증액을 요청하시거나 통신사의 결제 내용을 확인해주세요..
2.03775/0.02743: 140, 나이트크로우에 접속할 수 없어요.
	나이<em>트크</em>로<em>우는 온라인 환경에서 이용할 수 있는 </em>멀티<em> 플랫폼 게임이므</em>로,네트워크 환경이 좋지 않을 경우 게임 이용이 불가하거나 끊기는 현상이 발생할 수 있습니다. 쾌적한 게임 이용을 위해 통신 상태가 원활한 곳에서 플레이 해주시기 바랍니다.
1.83429/0.02028: 72, 퀘스트 진행 시 순간이동 방법을 알고싶어요.
	<em>퀘스트 목적지로 순간이동을 </em>원하시면 화면 우측 상단의<em> 퀘스트 목록에</em>서<em> 순간이동 아이콘을 클릭하</em>시<em>거나,</em> [메뉴] -<em> [퀘스트]탭에</em>서<em> 자동 이동 및 순간 이동이 가능합니다.</em>
1.79299/0.02963: 142, 나이트크로우 이용 중 강제 종료, 끊김, 지연 현상으로 이용이 어렵습니